## 1. 지하철역 위치정보를 기상청 격자(nx, ny) 로 변환
기상청 예보 데이터는 위도/경도(lat/lon)이 아닌 격자좌표계(nx,ny) 기준으로 제공되므로,
지하철역 위치를 예보 데이터와 연결하려면, 위도/경도 -> nx/ny 변환 작업선행 요구됨.

격자좌표 : 
| 항목 | 설명 |
|------|------|
| 좌표계 | LCC (Lambert Conformal Conic) 투영 기반 |
| 격자 간격 | 약 5km × 5km |
| 전체 격자 범위 | 동서 149 × 남북 253 (총 37,697개 격자) |
| 기준점 위경도 | 위도 38.0°, 경도 126.0° → nx: 43, ny: 136 |
| 사용 목적 | 위경도보다 간편한 기상자료 연계 계산 |

> 하나의 격자(nx, ny)는 약 5km 정방형 지역을 의미하며, 각 지점별 예보 및 관측 데이터가 이 단위로 제공됩니다.

### 1. 데이터 불러오기

In [25]:
import pandas as pd
import unicodedata

# 1. 파일 불러오기
main_df = pd.read_excel('지하철역_데이터_상세.xlsx')
subway1to8_df = pd.read_csv('raw/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv', encoding='cp949')
station_master_df = pd.read_csv('raw/서울시 역사마스터 정보.csv', encoding='cp949')

# 2. 특수문자 제거 및 정규화 함수 정의
def clean_name(name):
    if pd.isna(name):
        return ''
    name = str(name).strip().replace('\ufeff', '').replace('역', '')
    return unicodedata.normalize('NFKC', name)

# 3. 컬럼 이름 정리
main_df.columns = main_df.columns.str.strip()
subway1to8_df.columns = subway1to8_df.columns.str.strip()
station_master_df.columns = station_master_df.columns.str.strip()

# 4. 정제된 이름 컬럼 추가
main_df['지하철역명_clean'] = main_df['지하철역명'].apply(clean_name)
subway1to8_df['역명_clean'] = subway1to8_df['역명'].apply(clean_name)
station_master_df['역사명_clean'] = station_master_df['역사명'].apply(clean_name)

# 5. Step 1: 서울교통공사 파일 기준으로 위경도 병합
merged1 = pd.merge(
    main_df,
    subway1to8_df[['역명_clean', '위도', '경도']],
    how='left',
    left_on='지하철역명_clean',
    right_on='역명_clean',
    suffixes=('', '_공사')
)

# 6. fillna로 위경도 채우기
main_df['위도'] = main_df['위도'].fillna(merged1['위도_공사'])
main_df['경도'] = main_df['경도'].fillna(merged1['경도_공사'])

# 7. Step 2: 남은 결측값을 서울시 역사마스터 정보로 보완
merged2 = pd.merge(
    main_df,
    station_master_df[['역사명_clean', '위도', '경도']],
    how='left',
    left_on='지하철역명_clean',
    right_on='역사명_clean',
    suffixes=('', '_마스터')
)

main_df['위도'] = main_df['위도'].fillna(merged2['위도_마스터'])
main_df['경도'] = main_df['경도'].fillna(merged2['경도_마스터'])

# (선택) 결과 저장
# main_df.to_csv('지하철역_위경도_보완결과.csv', index=False)


In [68]:
main_df

,지하철역ID,지하철호선ID,역명,지하철호선명,환승역여부,지상/지하,이전지하철역ID,이전지하철역명,다음지하철역ID,다음지하철역명,...,구/동,환승호선수,지하철역X좌표,지하철역Y좌표,연계역번호,지하철역코드,급행정차여부(0:미정차/1:상행정차/2:하행정차/3:양방향정차),위도,경도,역명_clean
0,﻿1001000100,﻿1001,소요산,1호선,X,NaN,﻿1001001001,﻿청산,﻿1001000101,﻿동두천,...,상봉암동,﻿1,﻿205362.3891,﻿494245.6421,﻿100,﻿190,﻿3,37.187295,127.043287,소요산
1,﻿1001000101,﻿1001,동두천,1호선,X,NaN,﻿1001000100,﻿소요산,﻿1001000102,﻿보산,...,동두천동,﻿1,﻿204824.737,﻿492006.4241,﻿101,﻿189,﻿3,37.563793,126.810659,동두천
2,﻿1001000102,﻿1001,보산,1호선,X,NaN,﻿1001000101,﻿동두천,﻿1001000103,﻿동두천중앙,...,보산동,﻿1,﻿205021.9647,﻿490533.9431,﻿102,﻿188,﻿0,37.580861,126.895695,보산
3,﻿1001000103,﻿1001,동두천중앙,1호선,X,NaN,﻿1001000102,﻿보산,﻿1001000104,﻿지행,...,생연2동,﻿1,﻿204967.7474,﻿489106.9743,﻿103,﻿187,﻿3,37.538608,126.962537,동두천중앙
4,﻿1001000104,﻿1001,지행,1호선,X,NaN,﻿1001000103,﻿동두천중앙,﻿1001000105,﻿덕정,...,지행동,﻿1,﻿204914.1407,﻿488082.7877,﻿104,﻿186,﻿0,37.539274,126.961438,지행
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,﻿1093004018,﻿1093,달미,﻿서해선,X,NaN,﻿1093004017,﻿시흥능곡,﻿1093004019,﻿선부,...,단원구,﻿0,﻿183109.699,﻿427757.2089,﻿S24,﻿316,﻿0,37.594917,127.076116,달미
770,﻿1093004019,﻿1093,선부,﻿서해선,X,NaN,﻿1093004018,﻿달미,﻿1093004020,﻿초지,...,단원구,﻿0,﻿183163.4295,﻿426148.9937,﻿S25,﻿317,﻿0,NaN,NaN,선부
771,﻿1093004020,﻿1093,초지,﻿서해선,O,NaN,﻿1093004019,﻿선부,﻿1093004021,﻿시우,...,단원구,﻿3,﻿183000.4144,﻿424503.9334,﻿S26,﻿318,﻿0,37.599550,127.091909,초지
772,﻿1093004021,﻿1093,시우,﻿서해선,X,NaN,﻿1093004020,﻿초지,﻿1093004022,﻿원시,...,단원구,﻿0,﻿181894.037,﻿423789.3701,﻿S27,﻿319,﻿0,37.606596,127.107906,시우


In [31]:
merged2[['지하철역명_clean', '위도_마스터']].head()

,지하철역명_clean,위도_마스터
0,소요산,37.948100
1,동두천,37.927878
2,보산,37.913702
3,동두천중앙,37.901885
4,지행,37.892334


In [34]:
merged2[['지하철역명_clean', '경도_마스터']].head()

,지하철역명_clean,경도_마스터
0,소요산,127.061034
1,동두천,127.054790
2,보산,127.057277
3,동두천중앙,127.056482
4,지행,127.055716


In [35]:
# 1. 지하철역명_clean → 역명으로 이름 변경
main_df.rename(columns={'지하철역명_clean': '역명'}, inplace=True)

# 2. 기존 지하철역명 컬럼 삭제
main_df.drop(columns=['지하철역명'], inplace=True)

# 3. '지하철호선ID' 다음에 '역명' 컬럼 위치 재배치
cols = list(main_df.columns)
cols.remove('역명')
idx = cols.index('지하철호선ID')
cols.insert(idx + 1, '역명')
main_df = main_df[cols]

In [36]:
main_df

,지하철역ID,지하철호선ID,역명,지하철호선명,환승역여부,지상/지하,이전지하철역ID,이전지하철역명,다음지하철역ID,다음지하철역명,...,강남권/비강남권,구/동,환승호선수,지하철역X좌표,지하철역Y좌표,연계역번호,지하철역코드,급행정차여부(0:미정차/1:상행정차/2:하행정차/3:양방향정차),위도,경도
0,﻿1001000100,﻿1001,소요산,1호선,X,NaN,﻿1001001001,﻿청산,﻿1001000101,﻿동두천,...,-,상봉암동,﻿1,﻿205362.3891,﻿494245.6421,﻿100,﻿190,﻿3,37.948100,127.061034
1,﻿1001000101,﻿1001,동두천,1호선,X,NaN,﻿1001000100,﻿소요산,﻿1001000102,﻿보산,...,-,동두천동,﻿1,﻿204824.737,﻿492006.4241,﻿101,﻿189,﻿3,37.927878,127.054790
2,﻿1001000102,﻿1001,보산,1호선,X,NaN,﻿1001000101,﻿동두천,﻿1001000103,﻿동두천중앙,...,-,보산동,﻿1,﻿205021.9647,﻿490533.9431,﻿102,﻿188,﻿0,37.913702,127.057277
3,﻿1001000103,﻿1001,동두천중앙,1호선,X,NaN,﻿1001000102,﻿보산,﻿1001000104,﻿지행,...,-,생연2동,﻿1,﻿204967.7474,﻿489106.9743,﻿103,﻿187,﻿3,37.901885,127.056482
4,﻿1001000104,﻿1001,지행,1호선,X,NaN,﻿1001000103,﻿동두천중앙,﻿1001000105,﻿덕정,...,-,지행동,﻿1,﻿204914.1407,﻿488082.7877,﻿104,﻿186,﻿0,37.892334,127.055716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,﻿1093004018,﻿1093,달미,﻿서해선,X,NaN,﻿1093004017,﻿시흥능곡,﻿1093004019,﻿선부,...,-,단원구,﻿0,﻿183109.699,﻿427757.2089,﻿S24,﻿316,﻿0,37.594917,127.076116
770,﻿1093004019,﻿1093,선부,﻿서해선,X,NaN,﻿1093004018,﻿달미,﻿1093004020,﻿초지,...,-,단원구,﻿0,﻿183163.4295,﻿426148.9937,﻿S25,﻿317,﻿0,NaN,NaN
771,﻿1093004020,﻿1093,초지,﻿서해선,O,NaN,﻿1093004019,﻿선부,﻿1093004021,﻿시우,...,-,단원구,﻿3,﻿183000.4144,﻿424503.9334,﻿S26,﻿318,﻿0,37.599550,127.091909
772,﻿1093004021,﻿1093,시우,﻿서해선,X,NaN,﻿1093004020,﻿초지,﻿1093004022,﻿원시,...,-,단원구,﻿0,﻿181894.037,﻿423789.3701,﻿S27,﻿319,﻿0,37.606596,127.107906


In [37]:
print(main_df.columns.tolist())

['지하철역ID', '지하철호선ID', '역명', '지하철호선명', '환승역여부', '지상/지하', '이전지하철역ID', '이전지하철역명', '다음지하철역ID', '다음지하철역명', '우편번호', '기본주소', '상세주소', '도/시/군', '구', '강남권/비강남권', '구/동', '환승호선수', '지하철역X좌표', '지하철역Y좌표', '연계역번호', '지하철역코드', '급행정차여부(0:미정차/1:상행정차/2:하행정차/3:양방향정차)', '위도', '경도']


In [50]:
# 현재 어떤 컬럼명이 있는지 보고
main_df.columns

# 결측 데이터 조회 예시 (정확한 컬럼명에 맞춰 조정)
missing_final = main_df[main_df['위도'].isna() | main_df['경도'].isna()]
missing_final.head()

,지하철역ID,지하철호선ID,역명,지하철호선명,환승역여부,지상/지하,이전지하철역ID,이전지하철역명,다음지하철역ID,다음지하철역명,...,구/동,환승호선수,지하철역X좌표,지하철역Y좌표,연계역번호,지하철역코드,급행정차여부(0:미정차/1:상행정차/2:하행정차/3:양방향정차),위도,경도,역명_clean
82,﻿1001080158,﻿1001,세마,1호선,X,NaN,﻿1001080157,﻿병점,﻿1001080159,﻿오산대,...,세마동,﻿1,﻿203832.1303,﻿409777.6488,﻿1717,﻿22,﻿0,NaN,NaN,세마
413,﻿1009000903,﻿1009,공항시장,﻿9호선,X,NaN,﻿1009000902,﻿김포공항,﻿1009000904,﻿신방화,...,공항동,﻿1,﻿183251.399,﻿451590.5744,﻿903,﻿903,﻿0,NaN,NaN,공항시장
483,﻿1063075317,﻿1063,수색,﻿경의중앙선,X,NaN,﻿1063075318,﻿한국항공대,﻿1063075316,﻿디지털미디어시티,...,수색로,﻿1,﻿190785.0,﻿453490.8,﻿K317,﻿218,﻿0,NaN,NaN,수색
501,﻿1063075826,﻿1063,효창공원앞,﻿경의중앙선,O,NaN,﻿1063075312,﻿공덕,﻿1063075110,﻿용산,...,용문동,﻿2,﻿196585.8,﻿448892.9,﻿K826,﻿236,﻿3,NaN,NaN,효창공원앞
520,﻿1067080118,﻿1067,중랑,﻿경춘선,O,NaN,﻿1067080117,﻿회기,﻿1067080120,﻿상봉,...,중랑역로,﻿2,﻿206718.9,﻿455045.3,﻿P118,﻿192,﻿0,NaN,NaN,중랑


In [51]:
main_df[['역명_clean']].drop_duplicates().sort_values(by='역명_clean')

,역명_clean
742,4.19 민주묘지
9,가능
194,가락시장
66,가산디지털단지
417,가양
...,...
213,회현
703,효자
318,효창공원앞
429,흑석


In [52]:
# 누락값 보완
supplement_df=pd.read_excel('raw/지하철 위경도.xlsx')
supplement_df.colums = supplement_df.columns.str.strip() #공백 제거

C:\Users\somee\AppData\Local\Temp\ipykernel_11980\560447150.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  supplement_df.colums = supplement_df.columns.str.strip() #공백 제거


In [53]:
supplement_df

,id,name,address,roadAddress,displayCode,displayName,city_id,city_name,point_x,point_y,routeType_id,routeType_name,transfers
0,100,소요산,경기도 동두천시 상봉암동 126-3,경기도 동두천시 평화로 2925,100,소요산역,1000,서울,127.061049,37.948747,1,1호선,NaN
1,70134,노포,부산광역시 금정구 노포동 133,부산광역시 금정구 중앙대로 2238,134,노포역,7000,부산,129.094817,35.283594,71,1호선,NaN
2,40146,안심,대구광역시 동구 괴전동 536-1,대구광역시 동구 안심로 455,146,안심역,4000,대구,128.733868,35.871249,41,1호선,NaN
3,50129,평동,광주광역시 광산구 월전동 51-23,광주광역시 광산구 평동로 870,119,평동역,5000,광주,126.769559,35.124754,51,1호선,NaN
4,30122,반석,대전광역시 유성구 반석동 685,대전광역시 유성구 북유성대로 303,122,반석역,3000,대전,127.314616,36.392128,31,1호선,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1411,아산,충청남도 아산시 배방읍 장재리 268,충청남도 아산시 배방읍 희망로 90,P172,아산역,1000,서울,127.104573,36.792118,1,1호선,NaN
1054,1412,탕정,충청남도 아산시 탕정면 매곡리 472-1,충청남도 아산시 탕정면 매곡중앙6로 11,P173,탕정역,1000,서울,127.084638,36.788270,1,1호선,NaN
1055,1413,배방,충청남도 아산시 배방읍 구령리 148-2,충청남도 아산시 배방읍 온천대로 1967,P174,배방역,1000,서울,127.052842,36.777592,1,1호선,NaN
1056,1415,온양온천,충청남도 아산시 온천동 56-9,충청남도 아산시 온천대로 1496,P176,온양온천역,1000,서울,127.003178,36.780541,1,1호선,NaN


In [54]:
# 2. 역명 정제 함수 정의
def clean_name(name):
    if pd.isna(name):
        return ''
    name = str(name).strip().replace('\ufeff', '').replace('역', '')
    return unicodedata.normalize('NFKC', name)

# 3. 보완 파일에 정제된 역명 컬럼 생성
supplement_df['역명_clean'] = supplement_df['name'].apply(clean_name)

# 4. main_df에도 병합용 '역명_clean' 생성 (이미 역명이 정제되어 있다면 그대로 복사)
main_df['역명'] = main_df['역명'].astype(str)
main_df['역명_clean'] = main_df['역명'].apply(clean_name)

# 5. 위경도 누락된 행만 추출하고 인덱스 저장
missing_df = main_df[main_df['위도'].isna() | main_df['경도'].isna()].copy()
missing_df['원본인덱스'] = missing_df.index

# 6. 병합
merged = pd.merge(
    missing_df,
    supplement_df[['역명_clean', 'point_y', 'point_x']],
    how='left',
    on='역명_clean'
)

# 7. 보완값을 원본 main_df에 반영
main_df.loc[merged['원본인덱스'], '위도'] = merged['point_y']
main_df.loc[merged['원본인덱스'], '경도'] = merged['point_x']

In [56]:
main_df[main_df['위도'].isna() | main_df['경도'].isna()][['역명', '지하철호선명','위도','경도']]

,역명,지하철호선명,위도,경도
82,세마,1호선,NaN,NaN
413,공항시장,﻿9호선,NaN,NaN
483,수색,﻿경의중앙선,NaN,NaN
501,효창공원앞,﻿경의중앙선,NaN,NaN
520,중랑,﻿경춘선,NaN,NaN
532,마석,﻿경춘선,NaN,NaN
556,간석오거리,﻿인천1호선,NaN,NaN
568,지식정보단지,﻿인천1호선,NaN,NaN
587,청량리,﻿수인분당선,NaN,NaN
609,미금,﻿수인분당선,NaN,NaN


In [57]:
# 병합에 실패한 역 목록
nan_rows = main_df[main_df['위도'].isna() | main_df['경도'].isna()][['역명', '지하철호선명']]
print(nan_rows['역명'].isin(supplement_df['역명_clean']).value_counts())

역명
True    22
Name: count, dtype: int64


In [58]:
# 1. 누락된 행 추출 + 인덱스 저장
missing_df = main_df[main_df['위도'].isna() | main_df['경도'].isna()].copy()
missing_df['원본인덱스'] = missing_df.index

# 2. 병합
merged = pd.merge(
    missing_df,
    supplement_df[['역명_clean', 'point_y', 'point_x']],
    how='left',
    left_on='역명',
    right_on='역명_clean'
)

# 3. 결과 반영 (point_y → 위도, point_x → 경도)
main_df.loc[merged['원본인덱스'], '위도'] = merged['point_y']
main_df.loc[merged['원본인덱스'], '경도'] = merged['point_x']

In [70]:
main_df[main_df['위도'].isna() | main_df['경도'].isna()][['역명', '지하철호선명']]

,역명,지하철호선명
413,공항시장,﻿9호선
483,수색,﻿경의중앙선
501,효창공원앞,﻿경의중앙선
520,중랑,﻿경춘선
532,마석,﻿경춘선
556,간석오거리,﻿인천1호선
568,지식정보단지,﻿인천1호선
587,청량리,﻿수인분당선
609,미금,﻿수인분당선
614,신갈,﻿수인분당선


In [67]:
main_df.loc[main_df['역명_clean'] == '세마', '위도'] = 37.187295
main_df.loc[main_df['역명_clean'] == '세마', '경도'] = 127.043287

In [73]:
manual_coords = {
    '세마': (37.187295, 127.043287),
    '공항시장': (37.562434, 126.810678),
    '수색': (37.582336, 126.895563),
    '효창공원앞': (37.539729, 126.961705),
    '중랑': (37.595261, 127.077447),
    '마석': (37.653324, 127.311633),
    '간석오거리': (37.464351, 126.693103),
    '지식정보단지': (37.410568, 126.669881),
    '청량리': (37.580178, 127.046835),
    '미금': (37.350005, 127.108944),
    '신갈': (37.312212, 127.108882),
    '남동인더스파크': (37.407190, 126.695019),
    '신포': (37.456255, 126.705206),
    '양재시민의숲': (37.470050, 127.038315),
    '가정중앙시장': (37.524208, 126.675847),
    '주안': (37.464737, 126.679509),
    '부발': (37.260833, 127.490278),
    '백마': (37.658445, 126.794436),
    '김포공항': (37.562434, 126.801058),
    '신현': (37.456556, 126.793133),
    '시흥시청': (37.380251, 126.805312),
    '선부': (37.328414, 126.788541),
}


for name, (lat, lon) in manual_coords.items():
    main_df.loc[main_df['역명_clean'] == name, '위도'] = lat
    main_df.loc[main_df['역명_clean'] == name, '경도'] = lon

In [74]:
main_df[main_df['위도'].isna() | main_df['경도'].isna()][['역명', '지하철호선명']]

,역명,지하철호선명


In [76]:
main_df.head(10)

,지하철역ID,지하철호선ID,역명,지하철호선명,환승역여부,지상/지하,이전지하철역ID,이전지하철역명,다음지하철역ID,다음지하철역명,...,구/동,환승호선수,지하철역X좌표,지하철역Y좌표,연계역번호,지하철역코드,급행정차여부(0:미정차/1:상행정차/2:하행정차/3:양방향정차),위도,경도,역명_clean
0,﻿1001000100,﻿1001,소요산,1호선,X,NaN,﻿1001001001,﻿청산,﻿1001000101,﻿동두천,...,상봉암동,﻿1,﻿205362.3891,﻿494245.6421,﻿100,﻿190,﻿3,37.187295,127.043287,소요산
1,﻿1001000101,﻿1001,동두천,1호선,X,NaN,﻿1001000100,﻿소요산,﻿1001000102,﻿보산,...,동두천동,﻿1,﻿204824.737,﻿492006.4241,﻿101,﻿189,﻿3,37.563793,126.810659,동두천
2,﻿1001000102,﻿1001,보산,1호선,X,NaN,﻿1001000101,﻿동두천,﻿1001000103,﻿동두천중앙,...,보산동,﻿1,﻿205021.9647,﻿490533.9431,﻿102,﻿188,﻿0,37.580861,126.895695,보산
3,﻿1001000103,﻿1001,동두천중앙,1호선,X,NaN,﻿1001000102,﻿보산,﻿1001000104,﻿지행,...,생연2동,﻿1,﻿204967.7474,﻿489106.9743,﻿103,﻿187,﻿3,37.538608,126.962537,동두천중앙
4,﻿1001000104,﻿1001,지행,1호선,X,NaN,﻿1001000103,﻿동두천중앙,﻿1001000105,﻿덕정,...,지행동,﻿1,﻿204914.1407,﻿488082.7877,﻿104,﻿186,﻿0,37.539274,126.961438,지행
5,﻿1001000105,﻿1001,덕정,1호선,X,NaN,﻿1001000104,﻿지행,﻿1001000106,﻿덕계,...,덕정동,﻿1,﻿205446.067,﻿482653.5202,﻿105,﻿185,﻿3,37.595025,127.076661,덕정
6,﻿1001000106,﻿1001,덕계,1호선,X,NaN,﻿1001000105,﻿덕정,﻿1001000107,﻿양주,...,덕계동,﻿1,﻿204942.8611,﻿479879.9865,﻿106,﻿184,﻿0,37.594904,127.075833,덕계
7,﻿1001000107,﻿1001,양주,1호선,X,NaN,﻿1001000106,﻿덕계,﻿1001000108,﻿녹양,...,남방동,﻿1,﻿203948.0054,﻿474893.6035,﻿107,﻿183,﻿3,37.652799,127.311789,양주
8,﻿1001000108,﻿1001,녹양,1호선,X,NaN,﻿1001000107,﻿양주,﻿1001000109,﻿가능,...,녹양동,﻿1,﻿203724.6713,﻿473310.9602,﻿108,﻿182,﻿0,37.466909,126.707902,녹양
9,﻿1001000109,﻿1001,가능,1호선,X,NaN,﻿1001000108,﻿녹양,﻿1001000110,﻿의정부,...,의정부1동,﻿1,﻿203919.8,﻿472062.8,﻿109,﻿181,﻿0,37.378017,126.645426,가능


In [78]:
main_df.drop(columns=['역명_clean'], inplace=True)

In [80]:
# 1. 컬럼명 유니코드 제거
main_df.columns = main_df.columns.str.replace('\ufeff', '', regex=False).str.strip()

# 2. 문자열 데이터 전체에서 유니코드 제거
main_df = main_df.applymap(lambda x: x.replace('\ufeff', '') if isinstance(x, str) else x)


C:\Users\somee\AppData\Local\Temp\ipykernel_11980\3984440477.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  main_df = main_df.applymap(lambda x: x.replace('\ufeff', '') if isinstance(x, str) else x)


In [ ]:
main_df.to_csv('지하철역_위경도_추가.csv', index=False)